In [1]:
import cv2,os

data_path='/home/mini/rock-paper-scissor-vision/training_data/raw/train'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

{'rock': 0, 'paper': 1, 'scissor': 2}
['rock', 'paper', 'scissor']
[0, 1, 2]


In [2]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [3]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)
target=new_target

Using TensorFlow backend.


In [4]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
#model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(3,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [5]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [6]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=5,callbacks=[checkpoint],validation_split=0.2)

Train on 388 samples, validate on 98 samples
Epoch 1/5
388/388 [==============================] - 14s 37ms/step - loss: 0.9459 - accuracy: 0.5696 - val_loss: 0.6358 - val_accuracy: 0.7755
Epoch 2/5
388/388 [==============================] - 13s 33ms/step - loss: 0.4584 - accuracy: 0.8402 - val_loss: 0.2758 - val_accuracy: 0.9184
Epoch 3/5
388/388 [==============================] - 13s 33ms/step - loss: 0.2059 - accuracy: 0.9356 - val_loss: 0.0804 - val_accuracy: 0.9898
Epoch 4/5
388/388 [==============================] - 13s 33ms/step - loss: 0.0939 - accuracy: 0.9768 - val_loss: 0.0524 - val_accuracy: 0.9796
Epoch 5/5
388/388 [==============================] - 13s 33ms/step - loss: 0.0153 - accuracy: 0.9974 - val_loss: 0.0096 - val_accuracy: 1.0000


In [7]:
print(model.evaluate(test_data,test_target))

54/54 [==============================] - 0s 9ms/step
[0.03909760630793042, 0.9814814925193787]


In [8]:
labels_dict={0:'ROCK',1:'PAPER',2:'SCISSOR'}

In [22]:
import cv2 
import constant
cap=cv2.VideoCapture(0)

from random import random
c=random()

while(1):
    ret,frame=cap.read()
    frame=cv2.flip(frame,1)
    src=frame[100:200,100:200]
    cv2.rectangle(frame,(100,100),(200,200),(0,255,0),0)
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY) # convert to grayscale
    blur = cv2.blur(gray, (5, 5)) # blur the image
    ret, thresh = cv2.threshold(blur, 140, 255, cv2.THRESH_BINARY)
    
    reshaped=np.reshape(thresh,(1,100,100,1))
    result=model.predict(reshaped)
    label=np.argmax(result,axis=1)[0]
    
   
    if c<0.34:
        computer='rock'
        img=cv2.imread('/home/mini/Downloads/rock.jpg')
        img=cv2.resize(img,(200,200))
    elif c<=0.67:
        computer='paper'
        img=cv2.imread('/home/mini/Downloads/paper.jpeg')
        img=cv2.resize(img,(200,200))
    else:
        computer='scissors'
        img=cv2.imread('/home/mini/Downloads/scissors.jpeg')
        img=cv2.resize(img,(200,200))
    cv2.putText(frame,computer,(0,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
   
    
    if computer == 'scissors':
        if label==0:
            cv2.putText(frame,'User wins' ,(100,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
            frame[280:500,300:500]=img
        elif label==1:
            cv2.putText(frame,'Computer wins' ,(100,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
            frame[280:500,300:500]=img
        else:
            cv2.putText(frame,'Tie' ,(100,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
            frame[280:500,300:500]=img
                
    if computer == 'paper':
        if label==0:
            cv2.putText(frame,'Computer wins' ,(200,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
        elif label==1:
            cv2.putText(frame,'Tie' ,(200,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
        else:
             cv2.putText(frame,'User wins' ,(200,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
                
    if computer == 'rock':
        if label==0:
            cv2.putText(frame,'Tie' ,(200,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
        elif label==1:
            cv2.putText(frame,'User wins' ,(200,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
        else:
             cv2.putText(frame,'Computer wins' ,(200),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
    
            
    
    if label==0:
        cv2.putText(frame,'rock' ,(0,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
    elif label==1:
        cv2.putText(frame,'paper' ,(0,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
    elif label==2:
        cv2.putText(frame,'scissor' ,(0,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
    else:
        cv2.putText(frame,'something' ,(0,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3,cv2.LINE_AA)
        
    cv2.imshow('frame',frame)
    cv2.imshow('thresh',thresh)
    cv2.imshow('src',src)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
    

In [10]:
cap.release()

In [18]:
from random import random
print(random())

0.285331013460153
